# Fine Tuning Transformer for MultiClass Text Classification

In [1]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

/home/bombbom/Documents/Multil_Model_Detection/env-project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
import re

In [5]:
def remove_comments(string):
            pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
            # first group captures quoted strings (double or single)
            # second group captures comments (//single-line or /* multi-line */)
            regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
            def _replacer(match):
                # if the 2nd group is not None, then we have captured a real comment string.
                if match.group(2) is not None:
                    return "" 
                else: # otherwise, we will return the 1st group
                    return match.group(1) 
            return regex.sub(_replacer, string)

In [6]:
# m1 = pd.read_pickle("/home/bombbom/Documents/Multil_Model_Detection/dataset/reentrancy_train.pkl") # train
# m1 = pd.DataFrame(m1).drop(columns=['address','sourcecode_len','bytecode_len', 'dataset'])
# m1['slither'] = m1.slither.eq('reentrancy').mul(1)
# m1=m1.rename(columns={"slither":'label'})
# m1.source_code = m1.source_code.apply(remove_comments)
# m1['label'] = m1['label']*1

m2 = pd.read_pickle("/home/bombbom/Documents/Multil_Model_Detection/dataset/reentrancy_test.pkl") # val


m2 = pd.DataFrame(m2).drop(columns=['address','sourcecode_len','bytecode_len', 'dataset'])


m2['slither'] = m2.slither.eq('reentrancy').mul(1)


m2=m2.rename(columns={"slither":'label'})

m2.source_code = m2.source_code.apply(remove_comments)

m2['label'] = m2['label']*1

# m3 = pd.read_pickle("/home/bombbom/Documents/Multil_Model_Detection/dataset/reentrancy_val.pkl") # train
# m3 = pd.DataFrame(m3).drop(columns=['address','sourcecode_len','bytecode_len', 'dataset'])
# m3['slither'] = m3.slither.eq('reentrancy').mul(1)
# m3=m3.rename(columns={"slither":'label'})
# m3.source_code = m3.source_code.apply(remove_comments)
# m3['label'] = m3['label']*1

In [8]:
len(m1)

51197

In [7]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('/home/bombbom/Documents/Multil_Model_Detection/dataset/custom/')

In [8]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.source_code[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [9]:
# training_set = Triage(m1, tokenizer, MAX_LEN)
testing_set = Triage(m2, tokenizer, MAX_LEN)

In [12]:
# training_set

In [10]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

# training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [11]:
distil_model = DistilBertModel.from_pretrained("/home/bombbom/Documents/Multil_Model_Detection/dataset/pre_train/")
for prama in distil_model.parameters():
    prama.requires_grad=False

Some weights of the model checkpoint at /home/bombbom/Documents/Multil_Model_Detection/dataset/pre_train/ were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
distil_model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(52000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [12]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = distil_model
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [35]:
model = DistillBERTClass()
model.load_state_dict(torch.load(f="/home/bombbom/Documents/Multil_Model_Detection/notebook/Model_mlp_smart_bug5.pth"))

<All keys matched successfully>

In [36]:

model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [37]:
# Creating the loss function and optimizer
loss_function = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
from tqdm import tqdm

In [21]:
import evaluate

In [32]:
accuracy_metric = evaluate.load("accuracy")

In [23]:
len(training_loader)

1600

In [24]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch, name):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    accuracy = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        # print(len(data['ids']))
        # print(len(data))
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float32)
        # print(targets)
        # print(targets.size(0))
        outputs = model(ids, mask)
        loss = loss_function(torch.sigmoid(outputs),targets.view(-1,1))
        tr_loss += loss.item()
        
        pre = torch.round(torch.sigmoid(outputs))
        # print(pre)
        # print(targets.shape)
        # size = targets.shape
        # print(size)
        acc = accuracy_metric.compute(references=targets, predictions=pre.reshape(targets.shape))
        accuracy+=acc['accuracy']
        # print(accuracy)
        # print(acc)
        # print(pre.reshape(targets.shape).shape)
        # n_correct=acc['accuracy']*(len(data['ids']))
        # print(n_correct)
        # break
        # print(_)
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        # print(nb_tr_examples)
        if _%100==0:
            # loss_step = tr_loss/nb_tr_steps
            # accu_step = (n_correct*100)/nb_tr_examples 
            # print(f"Training Loss per 5000 steps: {loss_step}")
            accu_step = accuracy/nb_tr_steps

            print(f"Training Accuracy per 5000 steps: {accu_step}")
        # break
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        # break
    # print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    # epoch_loss = tr_loss/nb_tr_steps
    # epoch_accu = (n_correct*100)/nb_tr_examples
    # print(f"Training Loss Epoch: {epoch_loss}")
    epoch_accu = accuracy/len(training_loader)
    print(f"Training Accuracy Epoch: {epoch_accu}")
    path_name = "Model_rnn" + str(name) + ".pth"
    torch.save(obj=model.state_dict(),f=path_name) # only saving the state_dict() only saves the models learned parameters 
    return 

In [25]:
EPOCHS = 10

In [ ]:
for epoch in range(EPOCHS):
    train(epoch, epoch)
    

Training Accuracy per 5000 steps: 0.53125


## Test 

In [38]:
label =[]
predict = []
with torch.no_grad():
    for _, data in enumerate(testing_loader_wild, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float32)
        outputs = model(ids, mask)
        pre = torch.round(torch.sigmoid(outputs))
        label.extend(targets.cpu().numpy())
        predict.extend(pre.reshape(targets.shape).cpu().numpy())

In [39]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
clf_metrics.compute(references=label, predictions=predict)

{'accuracy': 0.8108945163053379,
 'f1': 0.6342494714587738,
 'precision': 0.7058823529411765,
 'recall': 0.5758157389635317}

## Trainning Smartbugs wild 


In [ ]:
from sklearn.model_selection import train_test_split

In [46]:
import pandas as pd
data = pd.read_pickle("/home/bombbom/Documents/Multil_Model_Detection/dataset/smart_bug_dataset.pkl")

In [47]:
data=data.rename(columns={'findings': 'label'})
data = data[((data['label']=='reentrancy') | (data['label']=='safe'))]
data['label'] = data.label.eq('reentrancy').mul(1)
data = data.reset_index(drop=True)

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
train, test = train_test_split(data,test_size=0.2, random_state=20)

In [51]:
train, test = train_test_split(train,test_size=0.2, random_state=20)

In [52]:
test

,source_code,label
24802,pragma solidity ^0.4.24;\n\n\n\ncontract Ownab...,1
20878,pragma solidity ^0.4.12;\n\n\nlibrary SafeMath...,0
9291,pragma solidity ^0.4.18;\n\ncontract ERC20Basi...,1
23613,pragma solidity ^0.4.25;\n\n\n\ncontract Multi...,0
7235,pragma solidity ^0.4.16;\n\ncontract Token {\n...,0
...,...,...
9015,pragma solidity ^0.4.18;\n\n\nlibrary SafeMath...,1
8982,pragma solidity ^0.4.18;\n\ncontract SafeMath ...,0
7165,pragma solidity ^0.4.24;\n\n\n\nlibrary SafeMa...,0
10571,pragma solidity ^0.5.8;\n\nlibrary SafeMath {\...,1


In [53]:
test[test['label']==1]

,source_code,label
24802,pragma solidity ^0.4.24;\n\n\n\ncontract Ownab...,1
9291,pragma solidity ^0.4.18;\n\ncontract ERC20Basi...,1
7969,pragma solidity ^0.4.24;\n\n\n\n\ncontract Own...,1
6464,pragma solidity ^0.4.6;\n\n\n\n\npragma solidi...,1
5438,pragma solidity 0.4.20;\n\n\n\n\nlibrary SafeM...,1
...,...,...
4531,pragma solidity ^0.4.18;\n\n\n\ninterface Tuli...,1
18166,pragma solidity 0.4.15;\n\nlibrary SafeMath {\...,1
12970,pragma solidity ^0.4.18;\n\n\nlibrary SafeMath...,1
9015,pragma solidity ^0.4.18;\n\n\nlibrary SafeMath...,1


In [54]:
test = test.reset_index(drop=True)

In [55]:
test

,source_code,label
0,pragma solidity ^0.4.24;\n\n\n\ncontract Ownab...,1
1,pragma solidity ^0.4.12;\n\n\nlibrary SafeMath...,0
2,pragma solidity ^0.4.18;\n\ncontract ERC20Basi...,1
3,pragma solidity ^0.4.25;\n\n\n\ncontract Multi...,0
4,pragma solidity ^0.4.16;\n\ncontract Token {\n...,0
...,...,...
4386,pragma solidity ^0.4.18;\n\n\nlibrary SafeMath...,1
4387,pragma solidity ^0.4.18;\n\ncontract SafeMath ...,0
4388,pragma solidity ^0.4.24;\n\n\n\nlibrary SafeMa...,0
4389,pragma solidity ^0.5.8;\n\nlibrary SafeMath {\...,1


In [56]:
testing_set_wild = Triage(test, tokenizer, MAX_LEN)

In [57]:
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
testing_loader_wild = DataLoader(testing_set_wild, **test_params)

In [58]:
label =[]
predict = []
with torch.no_grad():
    for _, data in enumerate(testing_loader_wild , 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float32)
        outputs = model(ids, mask)
        pre = torch.round(torch.sigmoid(outputs))
        label.extend(targets.cpu().numpy())
        predict.extend(pre.reshape(targets.shape).cpu().numpy())

In [59]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
clf_metrics.compute(references=label, predictions=predict)

{'accuracy': 0.8080163971760419,
 'f1': 0.6162949476558944,
 'precision': 0.6880081300813008,
 'recall': 0.5581203627370157}

In [30]:
def train(epoch, name):
    tr_loss = 0
    
    nb_tr_steps = 0
    nb_tr_examples = 0
    accuracy = 0
   
    model.train()
    for _,data in enumerate(testing_loader_wild, 0):
        # print(len(data['ids']))
        # print(len(data))
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float32)
        # print(targets)
        # print(targets.size(0))
        outputs = model(ids, mask)
        loss = loss_function(torch.sigmoid(outputs),targets.view(-1,1))
        tr_loss += loss.item()
        
        pre = torch.round(torch.sigmoid(outputs))

        acc = accuracy_metric.compute(references=targets, predictions=pre.reshape(targets.shape))
        accuracy+=acc['accuracy']

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        # print(nb_tr_examples)
        if _%100==0:

            accu_step = accuracy/nb_tr_steps

            print(f"Training Accuracy per {_} steps: {accu_step}")
        # break
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
    epoch_accu = accuracy/len(testing_set_wild)
    print(f"Training Accuracy Epoch: {epoch_accu}")
    path_name = "Model_mlp_smart_bug" + str(name+3) + ".pth"
    torch.save(obj=model.state_dict(),f=path_name) # only saving the state_dict() only saves the models learned parameters 
    return 

In [ ]:
for epoch in range(5):
    train(epoch, epoch)